# Models with Keras

Trains models on synthetic data .

In [4]:
import sys
sys.path.append('../..')


In [5]:
import tensorflow as tf
from numpy import arange
import tensorflow_datasets as tfds

import sys
sys.path.append('..')


In [6]:
from model_compra_comigo.experimenter import Experimenter
from model_compra_comigo.model.rnn.rnn_example_model import RNNExampleModel
from data_simulator import DataSimulator


ModuleNotFoundError: No module named 'model_compracomigo.model.rnn'

## Generate a Dataset

In [ ]:
data_simulator = DataSimulator()
time_range = 365*6
split = 365
time_data = arange(time_range)

series, parameters = data_simulator.generate(
    time_range = time_range,
)

labels=[(f"trend_slope = {parameters['trend_slope']}\n"
         f"seasonality_period = {parameters['seasonality_period']}\n"
         f"seasonality_amplitude = {parameters['seasonality_amplitude']}\n"
         f"seasonality_phase = {parameters['seasonality_phase']}\n"
         f"seasonality_time_threshold = {parameters['seasonality_time_threshold']}\n"
         f"seasonality_ncos = {parameters['seasonality_ncos']}\n"
         f"seasonality_nexp = {parameters['seasonality_nexp']}\n"
         f"noise_scaling_factor = {parameters['noise_scaling_factor']}\n"
         f"autocorrelation_amplitude = {parameters['autocorrelation_amplitude']}\n"
         f"autocorrelation_phi = {parameters['autocorrelation_phi']}"
        )]

data_simulator.plot_series(
    time=time_data,
    series=series,
    labels=labels
)


## Creates Windowed Dataset with N forecasts

In [ ]:
experimenter = Experimenter()

In [ ]:
window_size = 90
nforecast = 90
batch_size = 32
shuffle_buffer_size = 1000
epochs = 10
split = 0.8

In [ ]:
n_data = int(series.shape[0]*split)
n_data

In [ ]:
data_train = series[:n_data]
data_test = series[n_data:]

In [ ]:
data_train_windowed = experimenter.generate_windowed_dataset(
    data = data_train,
    window_size = window_size,
    batch_size = batch_size,
    shuffle_buffer_size = shuffle_buffer_size,
    nforecast = nforecast
)
data_test_windowed = experimenter.generate_windowed_dataset(
    data = data_test,
    window_size = window_size,
    batch_size = batch_size,
    shuffle_buffer_size = shuffle_buffer_size,
    nforecast = nforecast
)


In [ ]:
model = RNNExampleModel(
    window_size=window_size,
    nforecast=nforecast
)

In [ ]:
model.fit(
    data=data_train_windowed,
    epochs=epochs
)

In [ ]:
prediction = model.predict(
    data=data_test_windowed,
    window_size=window_size,
)

In [ ]:
prediction.shape

In [ ]:
import numpy
import numpy as np

In [ ]:
x_valid = rolling_window(data_test[window_size:], 90)
x_valid.shape

In [ ]:
((x_valid-prediction)**2).mean()

In [ ]:
tf.keras.metrics.mean_squared_error(x_valid, prediction).numpy().shape

In [ ]:
def rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

In [ ]:
data_test.shape

In [ ]:
data_test[window_size+1:].shape

In [ ]:
[x[0].shape for x in tfds.as_numpy(data_test)]

In [ ]:
print(tf.keras.metrics.mean_squared_error(x_valid, results).numpy())


In [3]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Conv1D

model = tf.keras.models.Sequential([
    Conv1D(
        filters=64, 
        kernel_size=3,
        strides=1,
        activation="relu",
        padding='causal',
        input_shape=[window_size, 1]
    ),
    LSTM(64, return_sequences=True),
    LSTM(64),
    Dense(30, activation="relu"),
    Dense(nforecasts),
])

model.summary()

NameError: name 'window_size' is not defined

In [ ]:
init_weights = model.get_weights()
len(init_weights), [init_weights[i].shape for i in range(0,len(init_weights))]

In [9]:
lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(epoch / 20))

# Initialize the optimizer
optimizer = tf.keras.optimizers.SGD(momentum=0.9)

# Set the training parameters
model.compile(loss=tf.keras.losses.Huber(), optimizer=optimizer)

# Train the model
history = model.fit(data, epochs=500, callbacks=[lr_schedule])

Epoch 1/500


2023-06-15 14:58:08.522048: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [2190]
	 [[{{node Placeholder/_0}}]]
2023-06-15 14:58:08.522239: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [2190]
	 [[{{node Placeholder/_0}}]]
2023-06-15 14:58:08.645642: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' 

1/1 [==============================] - 2s 2s/step - loss: 1000.7211 - lr: 1.0000e-08
Epoch 2/500
1/1 [==============================] - 0s 44ms/step - loss: 1000.7209 - lr: 1.1220e-08
Epoch 3/500
1/1 [==============================] - 0s 48ms/step - loss: 1000.7209 - lr: 1.2589e-08
Epoch 4/500
1/1 [==============================] - 0s 44ms/step - loss: 1000.7209 - lr: 1.4125e-08
Epoch 5/500
1/1 [==============================] - 0s 39ms/step - loss: 1000.7209 - lr: 1.5849e-08
Epoch 6/500
1/1 [==============================] - 0s 50ms/step - loss: 1000.7211 - lr: 1.7783e-08
Epoch 7/500
1/1 [==============================] - 0s 45ms/step - loss: 1000.7209 - lr: 1.9953e-08
Epoch 8/500
1/1 [==============================] - 0s 46ms/step - loss: 1000.7209 - lr: 2.2387e-08
Epoch 9/500
1/1 [==============================] - 0s 37ms/step - loss: 1000.7209 - lr: 2.5119e-08
Epoch 10/500
1/1 [==============================] - 0s 47ms/step - loss: 1000.7209 - lr: 2.8184e-08
Epoch 11/500
1/1 [=====

1/1 [==============================] - 0s 42ms/step - loss: 529.1594 - lr: 1.9953
Epoch 168/500
1/1 [==============================] - 0s 45ms/step - loss: 612.8081 - lr: 2.2387
Epoch 169/500
1/1 [==============================] - 0s 46ms/step - loss: 837.3863 - lr: 2.5119
Epoch 170/500
1/1 [==============================] - 0s 50ms/step - loss: 756.6285 - lr: 2.8184
Epoch 171/500
1/1 [==============================] - 0s 44ms/step - loss: 1216.2864 - lr: 3.1623
Epoch 172/500
1/1 [==============================] - 0s 45ms/step - loss: 1000.3276 - lr: 3.5481
Epoch 173/500
1/1 [==============================] - 0s 51ms/step - loss: 1000.2830 - lr: 3.9811
Epoch 174/500
1/1 [==============================] - 0s 43ms/step - loss: 1000.2006 - lr: 4.4668
Epoch 175/500
1/1 [==============================] - 0s 46ms/step - loss: 1000.0786 - lr: 5.0119
Epoch 176/500
1/1 [==============================] - 0s 40ms/step - loss: 999.9156 - lr: 5.6234
Epoch 177/500
1/1 [==============================

KeyboardInterrupt: 

In [ ]:
from matplotlib.pyplot import figure, grid, legend, plot, show, xlabel, ylabel, savefig, semilogx, tick_params, axis

# Define the learning rate array
lrs = 1e-8 * (10 ** (arange(500) / 20))

# Set the figure size
figure(figsize=(10, 6))

# Set the grid
grid(True)

# Plot the loss in log scale
semilogx(lrs[:500], history.history["loss"][:500])

# Increase the tickmarks size
tick_params('both', length=10, width=1, which='both')
# axis([1e-8, 1e4, 0, 500])

In [ ]:
model = tf.keras.models.Sequential([
    Conv1D(
        filters=64, 
        kernel_size=3,
        strides=1,
        activation="relu",
        padding='causal',
        input_shape=[window_size, 1]
    ),
    LSTM(64, return_sequences=True),
    LSTM(64),
    Dense(30, activation="relu"),
    Dense(nforecasts),
])

model.summary()



In [ ]:
lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(120. / 20.))

# Initialize the optimizer
optimizer = tf.keras.optimizers.SGD(momentum=0.9)

# Set the training parameters
model.compile(loss=tf.keras.losses.Huber(), optimizer=optimizer)

# Train the model
history = model.fit(data, epochs=500, callbacks=[lr_schedule])

In [ ]:
def model_forecast(model, series, window_size, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda w: w.batch(window_size))
    dataset = dataset.batch(batch_size).prefetch(1)
    forecast = model.predict(dataset)
    return forecast


In [ ]:
prediction = model_forecast(model, series, window_size, batch_size)


In [ ]:
type(prediction)

In [ ]:
forecast_series = series[0:window_size]
forecast = model_forecast(model, forecast_series, window_size, batch_size)
results = forecast.squeeze()


In [ ]:
time_pred = time_data[90:180]
image = plot_series(
    series = [(time_pred, results), (time_data[90:180], series[90:180]),],
    plt_labels = ["x", "y"],
    labels = ["line2", "line1"],
)

In [ ]:
window_size = 90
nforecasts = 90
batch_size = 32
shuffle_buffer_size = 1000

In [ ]:
data = experimenter.generate_windowed_dataset(
    data = series,
    window_size = window_size,
    batch_size = batch_size,
    shuffle_buffer_size = shuffle_buffer_size,
    nforecasts = nforecasts
)


In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Conv1D

model = tf.keras.models.Sequential([
    Conv1D(
        filters=64, 
        kernel_size=3,
        strides=1,
        activation="relu",
        padding='causal',
        input_shape=[window_size, 1]
    ),
    LSTM(64, return_sequences=True),
    LSTM(64),
    Dense(30, activation="relu"),
    Dense(nforecasts),
])
model.summary()

In [ ]:
# Define the scheduler
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    1.,
    decay_steps=500,
    decay_rate=0.99,
    staircase=True)

# Set the optimizer
optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9)

# Set the training parameters
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])

# Train the model
history = model.fit(data,epochs=2000,)

In [ ]:
prediction = model_forecast(model, series, window_size, batch_size)
forecast_series = series[0:window_size]
forecast = model_forecast(model, forecast_series, window_size, batch_size)
results = forecast.squeeze()
time_pred = time_data[90:180]
image = plot_series(
    series = [(time_pred, results), (time_data[90:180], series[90:180]),],
    plt_labels = ["x", "y"],
    labels = ["line2", "line1"],
)

## Lessons
Most promising approach, will need to see how to automate the selection of these hyperparameters (and make it robust in an application)